<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg" />
</center> 
     
## <center>  [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

Auteur: [Yury Kashnitsky](https://yorko.github.io) (@yorko). Edité et traduit par [Ousmane Cissé](https://github.com/oussou-dev). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose. 

# <center>Mission #2. Automne 2019
## <center> Partie 2. Gradient boosting

**Dans cette mission, vous êtes invité à faire mieux que la "baseline" (référence de base) dans la compétition ["Flight delays](https://www.kaggle.com/c/flight-delays-fall-2018).**

Cette fois-ci, nous avons décidé de partager un scénario de base CatBoost plutôt correct. Vous devrez améliorer la solution fournie.

Avant de vous lancer sur la mission, nous vous conseillons de consulter le matériel de cours correspondant:
 1. [Classification, arbres-de-décision et k-plus-proches-voisins](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic03_decision_trees_kNN/topic3_decision_trees_kNN.ipynb?flush_cache=true), version interactive sur [Kaggle](https://www.kaggle.com/kashnitsky/topic-3-decision-trees-and-knn)
 2. Ensembles:
  - [Bagging](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part1_bagging.ipynb?flush_cache=true), version interactive sur [Kaggle](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-1-bagging)
  - [Forêt aléatoire](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part2_random_forest.ipynb?flush_cache=true), version intercative sur [Kaggle](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-2-random-forest)
  - [Feature Importance](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part3_feature_importance.ipynb?flush_cache=true), version interactive sur [Kaggle](https://www.kaggle.com/kashnitsky/topic-5- ensembles-part-3-feature-importance)
 3. [Gradient boosting](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic10_boosting/topic10_gradient_boosting.ipynb?flush_cache=true), version interactive sur [Kaggle](https://www.kaggle.com/kashnitsky/topic-10-gradient-boosting)
  - Régression logistique, forêt aléatoire et LightGBM dans le cadre de la compétition "Kaggle Forest Cover Type Prediction": [Kernel](https://www.kaggle.com/kashnitsky/topic-10-practice-with-logit-rf-and-lightgbm) 
 4. Vous pouvez également vous exercer avec des missions de démonstration, plus simples et déjà partagées avec des solutions:
  - "Decision trees with a toy task and the UCI Adult dataset": [mission](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees) + [solution](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees-solution)
  - "Logistic Regression and Random Forest in the credit scoring problem": [mission](https://www.kaggle.com/kashnitsky/assignment-5-logit-and-rf-for-credit-scoring) + [solution](https://www.kaggle.com/kashnitsky/a5-demo-logit-and-rf-for-credit-scoring-sol)
 5. Il y a également 7 conférences vidéo sur les arbres, les forêts, la stimulation et leurs applications [mlcourse.ai/lectures](https://mlcourse.ai/lectures) 
 6. Tutoriels mlcourse.ai sur [categorical feature encoding](https://www.kaggle.com/waydeherman/tutorial-categorical-encoding) (par Wayde Herman) et [CatBoost](https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview) (par Mikhail Tribunskiy)
 7. Dernier point important : [Kernels publics](https://www.kaggle.com/c/flight-delays-fall-2018/notebooks) liés à cette compétition


### Votre mission consiste à:
 1. battre**"A2 de base (10 crédits)"**sur Public LB (**0.75914 <**> score LB)
 2. renommer votre [équipe](https://www.kaggle.com/c/flight-delays-fall-2018/team) en totale conformité avec A1 et le [classement du cours](https://docs.google.com/spreadsheets/d/15e1K0tg5ponA5R6YQkZfihrShTDLAKf5qeKaoVCiuhQ/) (à paraître le 16.09.2019)
 
Cette tâche est censée être relativement facile. Ici, vous n'êtes pas obligé de télécharger votre solution reproductible.
 
### <center> Date limite pour A2: le 6 octobre 2019 à 20h59 (heure de Londres)

In [1]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

**Lire les données**

In [2]:
PATH_TO_DATA = Path('../input/flight-delays-fall-2018/')

In [3]:
train_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_train.csv')

In [4]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [5]:
test_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_test.csv')

In [6]:
test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


**Créer une seule caractéristique - "flight" (que vous devez améliorer - ajouter plus de caractéristiques)**

In [7]:
train_df['flight'] = train_df['Origin'] + '-->' + train_df['Dest']
test_df['flight'] = test_df['Origin'] + '-->' + test_df['Dest']

**Se souvenir des index des caractéristiques catégorielles (à passer à CatBoost)**

In [8]:
categ_feat_idx = np.where(train_df.drop('dep_delayed_15min', axis=1).dtypes == 'object')[0]
categ_feat_idx

array([0, 1, 2, 4, 5, 6, 8])

**Allouer un jeu de données de test (par exemple, un jeu de validation) pour valider le modèle**

In [9]:
X_train = train_df.drop('dep_delayed_15min', axis=1).values
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test_df.values

In [10]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

**Entraîner Catboost avec les arguments par défaut, en ne passant que les index des caractéristiques catégorielles.**

In [11]:
ctb = CatBoostClassifier(random_seed=17, silent=True)

In [12]:
%%time
ctb.fit(X_train_part, y_train_part,
        cat_features=categ_feat_idx);

CPU times: user 5min 49s, sys: 42.3 s, total: 6min 32s
Wall time: 1min 44s


In [13]:
ctb_valid_pred = ctb.predict_proba(X_valid)[:, 1]

**Nous avons obtenu environ 0.756 ROC AUC sur les données de test (hold-out set).**

In [14]:
roc_auc_score(y_valid, ctb_valid_pred)

0.7558567262528356

**Entraîner l'ensemble des données d'entraînement, faites des prévisions sur l'ensemble des données de test. Nous obtenons ~ 0,734 lors de la compétition - Niveau de (baseline) référence "Catboost starter"**

In [15]:
%%time
ctb.fit(X_train, y_train,
        cat_features=categ_feat_idx);

CPU times: user 8min 3s, sys: 53.2 s, total: 8min 56s
Wall time: 2min 22s


In [16]:
ctb_test_pred = ctb.predict_proba(X_test)[:, 1]

In [17]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    sample_sub = pd.read_csv(PATH_TO_DATA / 'sample_submission.csv', 
                             index_col='id')
    sample_sub['dep_delayed_15min'] = ctb_test_pred
    sample_sub.to_csv('ctb_pred.csv')

In [18]:
!head ctb_pred.csv

id,dep_delayed_15min
0,0.03535583816921028
1,0.06161772607690032
2,0.044752068363054114
3,0.3298909878906396
4,0.3091382818085681
5,0.09379994946399604
6,0.06754366886182442
7,0.21169519883509133
8,0.1313800051843654


Maintenant c'est votre tour ! Lancez-vous et améliorez le modèle à battre **"A2 base de référence (10 crédits)"** - **0,75914** score LB. Il est crucial de proposer de bonnes caractéristiques.

Pour les discussions, reférez-vous en au fil **#a2_kaggle_fall2019** du canal **mlcourse_ai_news** [ODS Slack](http://opendatascience.slack.com). Serhii Romanenko (@serhii_romanenko) sera là pour vous aider.

Bienvenue sur Kaggle!

<img src='https://habrastorage.org/webt/fs/42/ms/fs42ms0r7qsoj-da4x7yfntwrbq.jpeg' width=50%>
*source ["Nerd Laughing Loud"](https://www.kaggle.com/general/76963).*